In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
all_date = session.query(Measurement.date).all()
for i in range(len(all_date) - 1):
    if all_date[i] <= all_date[i+1]:
        max_date = all_date[i+1]
    i = i+1
max_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.
print(max_date)

# Calculate the date one year from the last date in data set.
import datetime as dt
one_yr_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(one_yr_ago)


# Perform a query to retrieve the data and precipitation scores
one_yr_prcp = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= one_yr_ago).all()
one_yr_prcp


# Save the query results as a Pandas DataFrame and set the index to the date column
df_prcp = pd.DataFrame(one_yr_prcp, columns=['Date', 'Precipitation'])
df_prcp.set_index('Date', inplace = True)

# Sort the dataframe by date
df_prcp = df_prcp.sort_values('Date')
df_prcp.head()



# Use Pandas Plotting with Matplotlib to plot the data
fig, ax = plt.subplots(figsize = (20,16))

plt.title('12 Months of Precipitation Data')
plt.ylabel('Inches') 

df_prcp.plot(ax = ax)
plt.xticks(rotation = 90)
plt.savefig("Img/Precipitation.png", bbox_inches = 'tight')

plt.tight_layout()
plt.show()




In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df_prcp.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(func.count(Station.station))
total_stations.scalar()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_count = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
station_count

In [ ]:
# Find the most active stations (i.e. what stations have the most rows?)
most_active = station_count[0]
most_active

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
lowest_t = session.query(func.min(Measurement.tobs)).\
            filter(Measurement.station == 'USC00519281')
highest_t = session.query(func.max(Measurement.tobs)).\
            filter(Measurement.station == 'USC00519281')
avg_t = session.query(func.avg(Measurement.tobs)).\
            filter(Measurement.station == 'USC00519281')
lt = lowest_t.scalar()
ht = highest_t.scalar()
at = avg_t.scalar()

print(f"The lowest temperature for station USC00519281 is {lt}; \
The highest temperature for station USC00519281 is {ht}; \
            The average temperature for station USC00519281 is {at}.")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
one_yr_tobs = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.date >= one_yr_ago).\
    filter(Measurement.station == 'USC00519281').all()
one_yr_tobs

In [ ]:
# Save the query results as a Pandas DataFrame
df_tobs = pd.DataFrame(one_yr_tobs, columns=['Date', 'Temperature'])

In [ ]:
# Plot the results as a histogram
fig, ax = plt.subplots()
label_t = ['tobs']
df_tobs.plot.hist(bins = 12, ax=ax, label = label_t)
plt.xlabel('Temperature')
plt.legend(label_t)

plt.show()

# Close session

In [ ]:
# Close Session
session.close()